# imports

In [ ]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import seaborn as sns
import pandas as pd
import json
from matplotlib import pyplot as plt
from model import *
from plot_utils import *

In [ ]:
AGENT_COUNT = 248
EXPERIMENT_STEPS = 200_000
PERFORM_RUN = False
PERFORM_BUS_EXPERIMENTS = False
PERFORM_TOLL_EXPERIMENTS = False
DO_SHOW_PLOTS_IN_NOTEBOOK = True

In [ ]:
if DO_SHOW_PLOTS_IN_NOTEBOOK:
    plt.ion()
else:
    plt.ioff()

### Functions for saving and loading results

In [ ]:
def savexperiment_results(dataframe: pd.DataFrame, filename: str):
    dataframe.to_csv(filename)


def load_results(filename: str) -> pd.DataFrame:
    return pd.read_csv(filename, index_col=0)


def get_agent_histories(agents) -> list:
    return [agent.history for agent in agents]


def save_agent_histories(agents, filename: str):
    histories = get_agent_histories(agents)
    with open(filename, "w") as f:
        json.dump(histories, f)


def load_agent_histories(filename: str):
    with open(filename, "r") as f:
        return json.load(f)


def save_model(model, model_name: str):
    model_results = model.datacollector.get_model_vars_dataframe()
    model_agents = model.schedule.agents
    savexperiment_results(model_results, f"{model_name}_results.csv")
    save_agent_histories(model_agents, f"{model_name}_histories.json")


def load_model(model_name: str):
    return load_results(f"{model_name}_results.csv"), load_agent_histories(f"{model_name}_histories.json")

# Run the 4-link and 5-link models
save the results for later use

In [ ]:
if PERFORM_RUN:
    model_4link = Model_4link(AGENT_COUNT)
    model_4link.initialize_agents_positions()

    model_5link = Model_5link(AGENT_COUNT)
    model_5link.initialize_agents_positions()

    for _ in range(EXPERIMENT_STEPS):
        model_4link.step()
        model_5link.step()

    save_model(model_4link, "model_4link")
    save_model(model_5link, "model_5link")

In [ ]:
model_4link_results, model_4link_histories = load_model("model_4link")
model_5link_results, model_5link_histories = load_model("model_5link")

# Data analysis and visualization

In [ ]:
plot_travel_times(model_4link_results, "model_4link")

In [ ]:
plot_travel_times(model_5link_results, "model_5link")

## Find the final average travel times per route, as remembered by the agents
(for each agent, check their 30 round history, save the average travel time per route)

In [ ]:
final_average_travel_times(model_4link_histories)

In [ ]:
final_average_travel_times(model_5link_histories)

From these averages can be seen that for the 5-link system, everyone is worse off. Thus, in this case, adding the road gives rise to Braess paradox

In [ ]:
interpret_agent_strategies(model_4link_histories)

In [ ]:
interpret_agent_strategies(model_5link_histories)

# Plot the throughput of each route during training
throughput is plotted as the average number of cars that pass through a route per 1000 ticks, averaged over 3000 ticks 

In [ ]:
plot_throughput(model_4link_results, "model_4link")

In [ ]:
plot_throughput(model_5link_results, "model_5link")

In [ ]:
model_histories = [model_4link_histories, model_5link_histories]
model_names = ["4-link", "5-link"]
create_latex_table(model_histories, model_names)

# Bus lane experiments


# Experiment 1 - replace cars by busses in the 5 link model
Busses can only take route 153, cars are not allowed to take the buslane (route 153 )

In [ ]:
import os


def save_experiments(models: list[Model_5link_with_bus], model_names: list[str], folder_name: str):
    os.makedirs(folder_name, exist_ok=True)

    for i, model in enumerate(models):
        model_name = model_names[i]
        save_model(model, f"{folder_name}/{model_name}")


def load_experiments(folder_name: str):
    models = {}

    for file_name in os.listdir(folder_name):
        model_name = file_name.split("_")[0]
        if model_name not in models.keys():
            models[model_name] = load_model(f"{folder_name}/{model_name}")

    models = {k: v for k, v in sorted(models.items(), key=lambda item: float(item[0].split('=')[1]))}
    print("loaded models")
    return models


def save_bus_experiments(models: list[Model_5link_with_bus], model_names: list[str]):
    return save_experiments(models, model_names, "bus_experiments")


def load_bus_experiments():
    return load_experiments("bus_experiments")


def save_toll_experiments(models: list[Model_5link_with_bus], model_names: list[str]):
    return save_experiments(models, model_names, "toll_experiments")


def load_toll_experiments(subtract_toll=True):
    experiment_data = load_experiments("toll_experiments")
    if subtract_toll:
        for experiment_name, experiment_results in experiment_data.items():
            model_results, model_histories = experiment_results
            toll = experiment_name.split("=")[1]
            adjusted_histories = [
                [[route, time] if route != "R153" else [route, time - int(toll)]  for route, time in agent_history]
                for agent_history in model_histories
            ]
            experiment_data[experiment_name] = (model_results, adjusted_histories)
    return experiment_data


In [ ]:
def analyse_experiment(experiment_data, folder_name: str):
    """
    for each model:
     - plot travel times and throughput
     - collect final throughput
     - collect final average travel times
     - 
     
    """
    experiment_throughputs = {}
    travel_times = []

    for model_name, experiment_results in experiment_data.items():
        print(model_name)
        model_results, model_histories = experiment_results
        
        # plot travel times and throughput        
        plot_travel_times(model_results, f"{folder_name}/{model_name}")
        model_throughput = plot_throughput(model_results, f"{folder_name}/{model_name}")
        
        # collect avg throughput of last 1000 ticks/sweeps
        final_throughput = model_throughput.iloc[-1]["total throughput"]
        experiment_throughputs[model_name] = round(final_throughput)

        interpret_agent_strategies(model_histories)
        print()
        travel_times.append(final_average_travel_times(model_histories))
        print()
    
    print(travel_times)

    model_histories = [experiment_results[1] for experiment_results in experiment_data.values()]
    model_names = experiment_data.keys()
    
    model_names = [model_name.split('=')[1] for model_name in model_names]
    if folder_name == "bus_experiments":
        model_names = [int(float(name) * AGENT_COUNT) for name in model_names]
    elif folder_name == "toll_experiments":
        model_names = [int(name) for name in model_names]
    create_latex_table(model_histories, model_names)
    
    plt.figure()
    plt.plot(list(experiment_throughputs.keys()), list(experiment_throughputs.values()))
    # plt.show()
    
    print()


def analyse_bus_experiments():
    analyse_experiment(load_bus_experiments(), "bus_experiments")


def analyse_toll_experiments():
    analyse_experiment(load_toll_experiments(), "toll_experiments")

In [ ]:
bus_ratios = list(range(5, 30, 5))

for x in bus_ratios:
    ratio = x / 100
    n_bus = int(ratio * AGENT_COUNT)
    n_car = AGENT_COUNT - n_bus
    print(f"ratio: {ratio}. n_bus: {int(ratio * AGENT_COUNT)}. n_car: {n_car}")

    if PERFORM_BUS_EXPERIMENTS:
        bus_model = Model_5link_with_bus(n_car, n_bus)
        bus_model.initialize_agents_positions()

        for _ in range(EXPERIMENT_STEPS):
            bus_model.step()

        save_bus_experiments([bus_model], [f"{ratio=}"])

In [ ]:
analyse_bus_experiments()

# Run toll-road experiments

In [ ]:
toll_costs = [x for x in range(50, 501, 50)]
for toll in toll_costs:
    print(toll)
    if PERFORM_TOLL_EXPERIMENTS:
        toll_model = Model_5link_toll(AGENT_COUNT, toll)
        toll_model.initialize_agents_positions()

        for _ in range(EXPERIMENT_STEPS):
            toll_model.step()

        save_toll_experiments([toll_model], [f"{toll=}"])

In [ ]:
analyse_toll_experiments()